🛠 Use Case Overview: **Identify cybersecurity-related entities (e.g., malware names, IPs, attack types) from unstructured threat reports.**
- Goal: Detect and classify entities like IPs, malware names, and attack types in unstructured security text.
- Approach: Sequence labeling with LSTM (NER-style).
- Data: Small sample dataset with manually labeled tokens.

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, TimeDistributed, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


2025-08-01 17:38:02.093371: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 📄 Sample Data (Simulated)

In [6]:
texts = [
    "APT29 launched a phishing campaign from 198.51.100.12",
    "Emotet infected the system using a malicious PDF",
    "Connection established from 203.0.113.8 using CobaltStrike",
    "Trickbot used DNS tunneling to exfiltrate data",
    "Detected brute-force attack from 192.168.1.5",
    "A malicious script downloaded by DarkComet",
    "Suspicious connection to 45.33.32.156 with Meterpreter",
    "Cerber ransomware delivered via drive-by download",
    "SQL injection detected on login page",
    "Zeus exfiltrated banking credentials",
    "DDoS attack originated from 198.51.100.99",
    "RedLine stealer executed through email attachment",
    "Exploit attempt on Apache Struts via remote code execution",
    "SpyEye communicated with 104.20.0.1",
    "Ransomware activity observed from an unusual IP address 172.16.254.1",
    "Dropper detected using PowerShell payload",
    "Email contained malicious macro and linked to NetWire",
    "Recon activity from 203.0.113.45 targeting SSH",
    "Payload dropped by AgentTesla from host 198.18.0.1",
    "Keylogger activity linked to email campaign"
]

In [47]:
sentences = [
    [("APT29", "MALWARE"), ("launched", "O"), ("a", "O"), ("phishing", "ATTACK"), ("campaign", "O"), ("from", "O"), ("198.51.100.12", "IP")],
    [("Emotet", "MALWARE"), ("infected", "O"), ("the", "O"), ("system", "O"), ("using", "O"), ("a", "O"), ("malicious", "O"), ("PDF", "ATTACK")],
    [("Connection", "O"), ("established", "O"), ("from", "O"), ("203.0.113.8", "IP"), ("using", "O"), ("CobaltStrike", "MALWARE")],
    [("Trickbot", "MALWARE"), ("used", "O"), ("DNS", "ATTACK"), ("tunneling", "ATTACK"), ("to", "O"), ("exfiltrate", "O"), ("data", "O")],
    [("Detected", "O"), ("brute-force", "ATTACK"), ("attack", "ATTACK"), ("from", "O"), ("192.168.1.5", "IP")],
    [("A", "O"), ("malicious", "O"), ("script", "O"), ("downloaded", "O"), ("by", "O"), ("DarkComet", "MALWARE")],
    [("Suspicious", "O"), ("connection", "O"), ("to", "O"), ("45.33.32.156", "IP"), ("with", "O"), ("Meterpreter", "MALWARE")],
    [("Cerber", "MALWARE"), ("ransomware", "MALWARE"), ("delivered", "O"), ("via", "O"), ("drive-by", "ATTACK"), ("download", "ATTACK")],
    [("SQL", "ATTACK"), ("injection", "ATTACK"), ("detected", "O"), ("on", "O"), ("login", "O"), ("page", "O")],
    [("Zeus", "MALWARE"), ("exfiltrated", "O"), ("banking", "O"), ("credentials", "O")],
    [("DDoS", "ATTACK"), ("attack", "ATTACK"), ("originated", "O"), ("from", "O"), ("198.51.100.99", "IP")],
    [("RedLine", "MALWARE"), ("stealer", "MALWARE"), ("executed", "O"), ("through", "O"), ("email", "O"), ("attachment", "ATTACK")],
    [("Exploit", "O"), ("attempt", "O"), ("on", "O"), ("Apache", "O"), ("Struts", "O"), ("via", "O"), ("remote", "ATTACK"), ("code", "ATTACK"), ("execution", "ATTACK")],
    [("SpyEye", "MALWARE"), ("communicated", "O"), ("with", "O"), ("104.20.0.1", "IP")],
    [("Ransomware", "MALWARE"), ("activity", "O"), ("observed", "O"), ("from", "O"), ("an", "O"), ("unusual", "O"), ("IP", "O"), ("address", "O"), ("172.16.254.1", "IP")],
    [("Dropper", "MALWARE"), ("detected", "O"), ("using", "O"), ("PowerShell", "ATTACK"), ("payload", "O")],
    [("Email", "O"), ("contained", "O"), ("malicious", "O"), ("macro", "ATTACK"), ("and", "O"), ("linked", "O"), ("to", "O"), ("NetWire", "MALWARE")],
    [("Recon", "ATTACK"), ("activity", "O"), ("from", "O"), ("203.0.113.45", "IP"), ("targeting", "O"), ("SSH", "O")],
    [("Payload", "O"), ("dropped", "O"), ("by", "O"), ("AgentTesla", "MALWARE"), ("from", "O"), ("host", "O"), ("198.18.0.1", "IP")],
    [("Keylogger", "MALWARE"), ("activity", "O"), ("linked", "O"), ("to", "O"), ("email", "O"), ("campaign", "O")]
]

### 🧾 Preprocessing

In [91]:
# Extract words and tags
words = list(set([w for s in sentences for w, t in s]))
tags = list(set([t for s in sentences for w, t in s]))

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["PAD"] = 0
word2idx["UNK"] = 1
word2idx

{'address': 2,
 'IP': 3,
 'attempt': 4,
 'Email': 5,
 'RedLine': 6,
 'Meterpreter': 7,
 'banking': 8,
 'activity': 9,
 'malicious': 10,
 'phishing': 11,
 'a': 12,
 'from': 13,
 'SpyEye': 14,
 'infected': 15,
 'NetWire': 16,
 'script': 17,
 'Zeus': 18,
 'DDoS': 19,
 'PDF': 20,
 'originated': 21,
 'communicated': 22,
 '198.51.100.99': 23,
 'SSH': 24,
 'CobaltStrike': 25,
 'Dropper': 26,
 'drive-by': 27,
 '192.168.1.5': 28,
 'ransomware': 29,
 'injection': 30,
 'connection': 31,
 'Ransomware': 32,
 'PowerShell': 33,
 'system': 34,
 'via': 35,
 'the': 36,
 'A': 37,
 'established': 38,
 'delivered': 39,
 'Exploit': 40,
 'remote': 41,
 'tunneling': 42,
 'macro': 43,
 'page': 44,
 '45.33.32.156': 45,
 'brute-force': 46,
 '203.0.113.45': 47,
 'SQL': 48,
 'campaign': 49,
 'using': 50,
 'launched': 51,
 'login': 52,
 '104.20.0.1': 53,
 'linked': 54,
 'targeting': 55,
 'unusual': 56,
 'observed': 57,
 'Keylogger': 58,
 'download': 59,
 '198.51.100.12': 60,
 'AgentTesla': 61,
 'DarkComet': 62,
 'E

In [52]:
tag2idx = {t: i for i, t in enumerate(tags)}
tag2idx

{'O': 0, 'IP': 1, 'MALWARE': 2, 'ATTACK': 3}

In [54]:
# Convert sentences into index format
MAX_LEN = 10

X = [[word2idx.get(w, word2idx["UNK"]) for w, t in s] for s in sentences]
X = pad_sequences(X, maxlen=MAX_LEN, padding='post', value=word2idx["PAD"])

y = [[tag2idx[t] for w, t in s] for s in sentences]
y = pad_sequences(y, maxlen=MAX_LEN, padding='post', value=tag2idx["O"])
y = [to_categorical(i, num_classes=len(tags)) for i in y]

In [56]:
X

array([[ 99,  51,  12,  11,  49,  13,  60,   0,   0,   0],
       [ 63,  15,  36,  34,  50,  12,  10,  20,   0,   0],
       [ 80,  38,  13,  69,  50,  25,   0,   0,   0,   0],
       [ 88,  64,  82,  42,  75,  81,  85,   0,   0,   0],
       [ 89,  46,  87,  13,  28,   0,   0,   0,   0,   0],
       [ 37,  10,  17,  71,  72,  62,   0,   0,   0,   0],
       [ 70,  31,  75,  45,  97,   7,   0,   0,   0,   0],
       [ 96,  29,  39,  35,  27,  59,   0,   0,   0,   0],
       [ 48,  30,  84,  78,  52,  44,   0,   0,   0,   0],
       [ 18,  93,   8,  67,   0,   0,   0,   0,   0,   0],
       [ 19,  87,  21,  13,  23,   0,   0,   0,   0,   0],
       [  6,  77,  90,  91,  65,  94,   0,   0,   0,   0],
       [ 40,   4,  78, 102,  86,  35,  41,  66,  95,   0],
       [ 14,  22,  97,  53,   0,   0,   0,   0,   0,   0],
       [ 32,   9,  57,  13,  79,  56,   3,   2,  98,   0],
       [ 26,  84,  50,  33,  68,   0,   0,   0,   0,   0],
       [  5,  73,  10,  43,  74,  54,  75,  16,   0,   0

In [58]:
y

[array([[0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]]),
 array([[0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]]),
 array([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]]),
 array([[0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
      

### 📊 Define the LSTM Model

In [61]:
model = Sequential()
model.add(Embedding(input_dim=len(word2idx), output_dim=32, input_length=MAX_LEN, mask_zero=True))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tags), activation="softmax")))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ ?                      │   0 (unbuilt) │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### 🧪 Train Model

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

history = model.fit(np.array(X_train), np.array(y_train), batch_size=1, epochs=10, validation_data=(np.array(X_test), np.array(y_test)))

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9925 - loss: 0.0478 - val_accuracy: 1.0000 - val_loss: 0.0228
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9739 - loss: 0.1396 - val_accuracy: 1.0000 - val_loss: 0.0211
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9942 - loss: 0.0283 - val_accuracy: 1.0000 - val_loss: 0.0185
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0232 - val_accuracy: 1.0000 - val_loss: 0.0184
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0124 - val_accuracy: 1.0000 - val_loss: 0.0172
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0098 - val_accuracy: 1.0000 - val_loss: 0.0164
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0093 - val_accuracy: 1.0000 - val_loss: 0.0158
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0080 - val_accuracy: 1.0000 - val_loss

### 🔍 Prediction Function

In [98]:
idx2tag = {i: t for t, i in tag2idx.items()}
idx2word = {i: w for w, i in word2idx.items()}

def predict_entities(sentence):
    tokens = sentence.split()  # Add .lower() if needed!
    seq = [word2idx.get(w, word2idx["UNK"]) for w in tokens]
    seq_padded = pad_sequences([seq], maxlen=MAX_LEN, padding='post', value=word2idx["PAD"])
    pred = model.predict(seq_padded)[0]
    pred_tags = [idx2tag[np.argmax(p)] for p in pred][:len(tokens)]
    return list(zip(tokens, pred_tags))


### 🧾 Example Test

In [101]:
test_sentence = "APT29 used spearphishing from 198.51.100.12"
print(predict_entities(test_sentence))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[('APT29', 'MALWARE'), ('used', 'O'), ('spearphishing', 'O'), ('from', 'O'), ('198.51.100.12', 'IP')]
